 # Opis notatnika
 Ten notatnik jest kontunacją analizy danych o lotach i ich opóźnieniach. Od tego momentu zaczniemy łączyć posiadana przez nas zbiory danych, będąc w stanie dokonać dodatkowych analiz.

 Zanim jednak do tego przejdziemy, należy, podobnie jak w poprzednim kroku, skonfigurować odpowiednio notatnik.
 
 W tej części warsztatu ponownie wcielasz się w rolę Analiyka Danych, którego zadaniem jest wykonanie analizy eksplotacyjnej zbioru danych - jedno z wymagań dostarczonych przez klienta.

 Tutaj zaimportuj wymagane biblioteki

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import plotly.graph_objects as go
import plotly.express as px

 ## Połączenie z bazą danych
 Tutaj uzupełnij konfigurację połączenia

In [2]:
username = 'postgres'
password = 'Arch3olog!A'  #stworzyć plik, zaczytać dane z pliku

host = 'localhost'
database = 'airlines_data'
port = 5432

 Tutaj stwórz zmienną engine, która zostanie użyta do połączenia z bazą danych

In [3]:
url = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(url)

 Tutaj uzupełnij implementację metody `read_sql_table`

In [4]:
def read_sql_table(table_name):
    raw_data = pd.DataFrame()
    for chunk in pd.read_sql_table(table_name, engine, chunksize=100000):
        raw_data = pd.concat([raw_data, chunk])
    return raw_data

 Tutaj zaczytaj zapisaną wcześniej ramkę danych `flight_df` do zmniennej o takiej samej nazwie

In [5]:
flight_df = pd.read_csv(r'../data/processed/flight_df_01.csv')

In [6]:
flight_df.head()

,id,month,day_of_month,day_of_week,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,...,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,year,is_delayed,is_weekend,distance_agg
0,1,1,20,7,WN,N204WN,682,10397,11292,605,...,5,NaN,NaN,NaN,NaN,NaN,2019,0,1,"(1100, 1200]"
1,2,1,20,7,WN,N8682B,2622,10397,11292,2120,...,5,NaN,NaN,NaN,NaN,NaN,2019,0,1,"(1100, 1200]"
2,3,1,20,7,WN,N717SA,2939,10397,11292,1800,...,5,4.0,0.0,10.0,0.0,3.0,2019,0,1,"(1100, 1200]"
3,4,1,20,7,WN,N709SW,3848,10397,11292,1355,...,5,NaN,NaN,NaN,NaN,NaN,2019,0,1,"(1100, 1200]"
4,5,1,20,7,WN,N7864B,1352,10397,11697,1125,...,3,NaN,NaN,NaN,NaN,NaN,2019,0,1,"(500, 600]"


Sprawdzenie poprawności danych w ramce `flight_df` 

In [7]:
flight_df_expected_rows_amount = 1057391
flight_df_rows_amount = flight_df.shape[0]

assert flight_df_rows_amount == flight_df_expected_rows_amount, f'Oczekiwano {flight_df_expected_rows_amount} wierszy, otrzymano {flight_df_rows_amount}'

 # Wzbogacenie o `aircraft`
 Używając procedury `read_sql_table` wczytaj dane z tabeli `aircraft` i zapisz jako `aircraft_df`. Następnie:  
 1. Usuń z ramki kolumny `number_of_seats` oraz `id`. Na tej podstawie usuń nadmiarowe wiersze (duplikaty).  
 1. Następnie jeszcze raz sprawdź, czy dla kolumny `tail_num` nie występują duplikaty. Innymi słowy należy sprawdzić, czy dla jednego `tail_num` występuje więcej niż jeden rok produkcji.  
 1. Jeśli tak to:  
     - do ramki `aircraft_df_duplicated` zapisz powielone zgodnie ze sprawdzeniem wiersze,  
     - zgodnie z powyższym zmodyfikuj ramkę tak, aby w przypadku duplikatu za datę wytworzenia samolotu, uznana została najnowsza tj. jeśli dla `tail_num` są dostępne daty produkcji 1998 oraz 2001, uznajemy, że `tail_num` został wyprodukowany w `2001`.

 Wskazówki:
 - Praca z duplikatami na LMS: `Python - analiza danych -> Dzień 5 - Pandas -> Duplikaty`
 - Dokumentacja metody `duplicated`: [klik](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html)
 - Dokumentacja metody `drop_duplicates`: [klik](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

 Tutaj wczytaj tabelę `aircraft` używając `read_sql_table`

In [8]:
aircraft_df = read_sql_table('aircraft')

In [9]:
aircraft_df.head()

,id,manufacture_year,tail_num,number_of_seats
0,7384,1944,N54514,0.0
1,7385,1945,N1651M,0.0
2,7386,1953,N100CE,0.0
3,7387,1953,N141FL,0.0
4,7388,1953,N151FL,0.0


 Tutaj usuń kolumny `number_of_seats`, `id` oraz duplikaty z ramki `aircraft_df`

In [10]:
aircraft_df.drop(columns = ['id', 'number_of_seats'], inplace = True)

In [11]:
aircraft_df.duplicated(keep = 'first').value_counts()  # sprawdzam ilość zduplikowanych wierszy

False    7364
True       19
Name: count, dtype: int64

In [12]:
aircraft_df.drop_duplicates(keep = 'first', inplace = True)

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [13]:
aircraft_df_expected_rows = 7364
aircraft_df_expected_columns = set(['tail_num', 'manufacture_year'])

aircraft_df_rows = aircraft_df.shape[0]

diff = aircraft_df_expected_columns.symmetric_difference(set(aircraft_df.columns))
assert aircraft_df_rows == aircraft_df_expected_rows, f'Spodziewano się {aircraft_df_expected_rows} wierszy , otrzymano {aircraft_df_rows} wierszy'

assert diff == set([]), f'Spodziewano się {aircraft_df_expected_columns} kolumn, otrzymano: {aircraft_df_expected_columns} kolumn. Różnica: \n\t{diff}'

 Tutaj sprawdź czy w ramkce `aircraft_df` występują duplikaty wewnątrz kolumny `tail_num`. Czyli czy dla danego `tail_num` występuje więcej niż jeden rok produkcji.

In [14]:
aircraft_df_is_duplicated = aircraft_df.duplicated(subset='tail_num')
aircraft_df_duplicated = aircraft_df.loc[aircraft_df_is_duplicated]

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić czy ta część została poprawnie wykonana

In [15]:
aircraft_df_expected_rows = 3
aircraft_df_duplicated_rows = aircraft_df_duplicated.shape[0]
assert aircraft_df_duplicated_rows == aircraft_df_expected_rows, f"Oczekiwano {aircraft_df_expected_rows} wierszy, otrzymano {aircraft_df_duplicated_rows}"

 ## Modyfikacja `aircraft_df`
 Tutaj dokonaj aktualizacji tabeli `aircraft_df` - jeśli jest taka potrzeba. Zrób to tak aby, dla powielonych `tail_num`, `manufacture_year` został ustawiony jako najwyższy

In [16]:
# sprawdzam zduplikowane numery na ogonie
duplicated_rows = aircraft_df[aircraft_df.duplicated(subset='tail_num', keep=False)]
print(duplicated_rows)

      manufacture_year tail_num
1734              1999   N783CA
2086              2000   N783CA
2460              2001   N877AS
4917              2011   N856GT
5725              2014   N856GT
6746              2017   N877AS


In [17]:
aircraft_df.loc[duplicated_rows.index, 'manufacture_year'] = duplicated_rows.groupby('tail_num')['manufacture_year'].transform('max')

In [18]:
aircraft_df.query("tail_num == 'N783CA'")  # sprawdzam wykonanie

,manufacture_year,tail_num
1734,2000,N783CA
2086,2000,N783CA


In [19]:
aircraft_df.drop_duplicates(keep = 'first', inplace = True)

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [20]:
test_tail = 'N783CA'
test_value = aircraft_df.loc[aircraft_df['tail_num']
                             == test_tail]['manufacture_year']
test_value = int(test_value)

expected_value = 2000
assert test_value == expected_value, f"Dla 'tail_num' == '{test_tail}' oczekiwano {expected_value} otrzymano {test_value}"

C:\Users\ankaz\AppData\Local\Temp\ipykernel_11244\1562451788.py:4: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  test_value = int(test_value)


 ## Połączenie `aircraft_df` oraz `flight_df`
 Połącz ramkę `aircraft_df` oraz `flight_df` zgodnie z kluczem oraz odpowiedz na następujące pytania:
 1. Czy po połączeniu pojawiły się duplikaty? Dokonaj odpowiedniego sprawdzenia.
 1. Wyznacz zależność roku produkcji do częstotliwości opóźnień. Wynik zapisz do tabeli `delays_by_manufacture_year_df`.
 1. Przedstaw wyniki w tabeli za pomocą wykresu punktowego.
 1. Dokonaj modyfikacji w taki sposób, aby wyświetlone na wykresie zostały tylko takie roczniki samolotów, które wykonały łącznie więcej niż 10000 `(>)` lotów.

> **Wskazówka:**
> Aby nie utracić potencjalnie całej dotychczasowej pracy, zapisz wynik do tymczasowej zmiennej np. `tmp_flight_df`. Po sprawdzeniu możesz użyć metody `copy`: [link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html), aby nadpisać `flight_df` używając `tmp_flight_df`.

 Tutaj dokonaj złączenia `flight_df` z `aircraft_df`, wynik zapisz do `tmp_flight_df`

In [21]:
tmp_flight_df = flight_df.merge(aircraft_df, on = 'tail_num', how = 'left' )

 Tutaj dokonaj sprawdzenia, czy nie pojawiły się duplikaty

In [22]:
temp_duplicated_rows = tmp_flight_df[tmp_flight_df.duplicated()]
print(temp_duplicated_rows)  #sprawdzam czy są zduplikowane wiersze

Empty DataFrame
Columns: [id, month, day_of_month, day_of_week, op_unique_carrier, tail_num, op_carrier_fl_num, origin_airport_id, dest_airport_id, crs_dep_time, dep_time, dep_delay, dep_time_blk, crs_arr_time, arr_time, arr_delay_new, arr_time_blk, cancelled, crs_elapsed_time, actual_elapsed_time, distance, distance_group, carrier_delay, weather_delay, nas_delay, security_delay, late_aircraft_delay, year, is_delayed, is_weekend, distance_agg, manufacture_year]
Index: []

[0 rows x 32 columns]


 Tutaj nadpisz `flight_df` używając `tmp_flight_df`

In [23]:
flight_df = tmp_flight_df.copy()

In [24]:
flight_df.shape  

(1057391, 32)

 ## Opóźnienia vs. rok produkcji samolotu cz. 1
 Tutaj wyznacz zależność roku produkcji do częstotliwości opóźnień

In [25]:
delays_by_manufacture_year_df = round(flight_df.groupby('manufacture_year')['is_delayed'].agg(['mean', 'count']), 2).reset_index()

In [26]:
delays_by_manufacture_year_df.head()

,manufacture_year,mean,count
0,1987.0,0.13,443
1,1988.0,0.18,634
2,1989.0,0.15,13
3,1990.0,0.16,4786
4,1991.0,0.18,7701


 Tutaj wyrysuj ramkę `delays_by_manufacture_year_df`

In [27]:
color_list = ["#bbe1c3","#a7cdbd","#869d7a","#8c8b6c","#91785d","#8e6b48","#8b5d33"]

In [28]:
fig_1 = px.scatter(
    delays_by_manufacture_year_df,
    x = 'manufacture_year',
    y = 'mean',
    color = 'mean',
    color_continuous_scale=color_list,
    template = 'plotly_white',
    size='count',
    title = "Percentage of delays in relation to aircraft production year",
    labels = {'mean': "Delayed flights (%)", 'manufacture_year': "Manufacture year", 'count': "Flights amount"}

)

fig_1.show()

 Tutaj zmodyfikuj wykres tak, aby prezentował tylko te roczniki, które odbyły więcej niż 10000 lotów

In [29]:
delays_by_manufacture_year_filtered_df = delays_by_manufacture_year_df[delays_by_manufacture_year_df['count']  > 10000]
fig_2 = px.scatter(
    delays_by_manufacture_year_filtered_df,
    x = 'manufacture_year',
    y = 'mean',
    color = 'mean',
    color_continuous_scale=color_list,
    template = 'plotly_white',
    size='count',
    title = "Percentage of delays bs year of production for over 10,000 flights",
    labels = {'mean': "Delayed flights (%)", 'manufacture_year': "Manufacture year", 'count': "Flights amount"}
)

fig_2.show()

 ## Opóźnienia vs. rok produkcji samolotu cz. 2
 Dokonaj agregacji kolumny `manufacture_year` do kolumny `manufacture_year_agg` zgodnie z poniższym:
 1. Grupując dane co 3 lata -> Czy po grupowaniu można zauważyć zależność? Wyniki zapisz do ramki `flight_delays_by_manufacture_year_agg_df`.
 1. Wyznacz top 5 roczników samolotu, które wykonały najwięcej lotów. Wyniki zapisz do ramki `top_manufactured_df`, do obliczeń wykorzystaj `delays_by_manufacture_year_df`.

 Tutaj dodaj kolumnę `manufacture_year_agg` do ramki `flight_df`

In [30]:
flight_df['manufacture_year'].unique()

array([2005., 2016., 1998., 2001., 2007., 2000., 2004., 1999., 2017.,
       2018., 2013., 2002., 2009., 2008.,   nan, 2011., 2006., 2014.,
       2015., 2003., 2010., 2012., 1997., 1993., 1995., 1992., 1996.,
       1991., 1990., 1988., 1987., 1994., 2019., 1989.])

In [31]:
flight_df['manufacture_year'].isna().sum()

12217

In [32]:
min_year = int(flight_df['manufacture_year'].min())
max_year = int(flight_df['manufacture_year'].max())

 Tutaj stwórz zmienną `flight_delays_by_manufacture_year_agg_df`

In [33]:
flight_df['manufacture_year_agg'] = pd.cut(flight_df['manufacture_year'], bins = range(min_year, max_year, 3), right = False).astype(str)

In [34]:
flight_delays_by_manufacture_year_agg_df = round(flight_df.groupby('manufacture_year_agg')['is_delayed'].agg(['mean', 'count'], ), 2).reset_index()
flight_delays_by_manufacture_year_agg_df.head()

,manufacture_year_agg,mean,count
0,"[1987, 1990)",0.16,1090
1,"[1990, 1993)",0.17,26369
2,"[1993, 1996)",0.20,18484
3,"[1996, 1999)",0.19,61624
4,"[1999, 2002)",0.17,202484


 Tutaj stwórz wykres w oparciu o dane zawarte w `flight_delays_by_manufacture_year_agg_df`

In [35]:
fig_3 = px.scatter(
    flight_delays_by_manufacture_year_agg_df,
    x = 'manufacture_year_agg',
    y = 'mean',
    color = 'mean',
    color_continuous_scale=color_list,
    template = 'plotly_white',
    title = "Percentage of delays in relation to aircraft production years (groups)",
    labels = {'mean': "Delayed flights (%)", 'manufacture_year_agg': "Manufacture year group",}
)
fig_3.update_xaxes(tickangle=-45)
fig_3.update_traces(marker_size = 10)

fig_3.show()

Tutaj wyznacz TOP 5 roczników produkcji - czyli sortując według liczby wykonanych lotów, pamiętaj o wyświetleniu również wartości opóźnienia.

In [36]:
sorted_manufactured_df = delays_by_manufacture_year_df.sort_values(by = 'count', ascending = False)
sorted_manufactured_df['manufacture_year'].astype(int)
top_manufactured_df = sorted_manufactured_df.head(5)
top_manufactured_df
# wartość opóźnienia jest jako 'mean'

,manufacture_year,mean,count
14,2001.0,0.15,100251
29,2016.0,0.19,66191
30,2017.0,0.19,62353
27,2014.0,0.18,61128
28,2015.0,0.19,58426


 # Podsumowanie
 W tym notatniku do naszej wyjściowej ramki danych `flight_df` dołączyliśmy tabelę `aircraft_df` i za jej pomocą dodaliśmy kolejny wymiar do naszej analizy. Zauważmy, ile dodatkowych wniosków mogliśmy wyciągnąć dzięki jej dodaniu.

 Zanim przejdziemy dalej, należy zapisać bieżącą postać ramki (najlepiej lokalnie), która zostanie użyta w kolejnym notatniku.

 > **Wskazówka:**  
 > Aby uniknąć potencjalnych problemów, najlepiej zapisać ramkę z nazwą nawiązującą do tego notatnika, np. `flight_df_01`.

 Tutaj zapisz ramkę w najdogodniejszy sposób

In [37]:
file_path = (r'../data/processed/flight_df_02.csv')

flight_df.to_csv(file_path, index = False)